In [1]:
import sys
sys.path.append('/Users/dan/Code/LLM/llm_researcher')

In [6]:
from llm_researcher.config.config import Config
config = Config()
print(config.search_retriever)
print(config.scraper)

<function tavily_search at 0x105b49940>
bs_scraper


In [3]:
sub_query = "Burning Man 2024 flood incident report"
search_results = config.search_retriever(sub_query,max_results=config.max_search_results_per_query)

search_results

[{'href': 'https://www.foxweather.com/weather-news/burning-man-festival-nevada-death-flooding-monsoon',
  'body': "BLACK ROCK CITY, Nev. – Tens of thousands of people who were attending the Burning Man festival in remote western Nevada remain stranded and have been told to shelter in place after monsoon rains led to flash flooding, and local authorities confirmed that they are investigating at least one death that was reported during the rain event.\n Death reported at Burning Man festival in Nevada as thousands of attendees get stranded by monsoon flooding\nEvent organizers told the more than 73,000 people attending the Burning Man festival to hunker down, conserve resources and prepare for more rain and flooding.\n The organizers of the Burning Man festival encouraged those who were attending the festival to bring goggles or face coverings due to the extreme weather conditions that have caused issues with blowing sand and dust in the past.\n LEAVES TENS OF THOUSANDS STRANDED IN MUDDY

In [4]:
new_search_urls =[url.get("href") for url in search_results]
new_search_urls

['https://www.foxweather.com/weather-news/burning-man-festival-nevada-death-flooding-monsoon',
 'https://apnews.com/article/burning-man-festival-flooding-entrance-closed-d6cd88ee009c6e1f6d2d92739ec1ca18',
 'https://www.foxweather.com/weather-news/burning-man-festival-mud-rain-desert',
 'https://www.pbs.org/newshour/nation/death-at-burning-man-festival-under-investigation-as-flooding-strands-thousands',
 'https://www.washingtonpost.com/climate-environment/2023/09/05/burning-man-exodus-flooding-climate-change/']

In [5]:
from concurrent.futures.thread import ThreadPoolExecutor
import llm_researcher.scraper.scraper_methods
def extract_data_from_link(scraper_nm, link):
    """
    Extracts the data from the link
    """
    content = ""
    try:
        scraper = getattr(llm_researcher.scraper.scraper_methods, scraper_nm)
        content = scraper(link)

        if len(content) < 100:
            return {"url": link, "raw_content": None}
        return {"url": link, "raw_content": content}
    except Exception as e:
        return {"url": link, "raw_content": None}



In [7]:
config = Config()
url = "https://www.foxweather.com/weather-news/burning-man-festival-nevada-death-flooding-monsoon"
data_from_link = extract_data_from_link(config.scraper,url)
data_from_link

{'url': 'https://www.foxweather.com/weather-news/burning-man-festival-nevada-death-flooding-monsoon',
 'raw_content': '\nDeath reported at Burning Man festival in Nevada as thousands of attendees get stranded by monsoon flooding | Fox Weather\n  Search Fox Weather  The FOX Weather App Learn More   \n              Watch Live\n            \n              Weather News\n            \n              How to Watch\n            \n              Extreme Weather\n            \n              Learn\n            \n              Local Weather\n            \n              Earth & Space\n            \n              Lifestyle\n            \n              Video\n            \n              America\'s Weather Team\n            \n              Podcast\n             Weather News  Published  \n        September 3, 2023 2:55pm EDT\n       \n            Death reported at Burning Man festival in Nevada as thousands of attendees get stranded by monsoon flooding\n           \n            Event organizers told the 

In [8]:

def run():
    with ThreadPoolExecutor(max_workers=20) as executor:
        contents = executor.map(lambda url: extract_data_from_link(config.scraper, url), new_search_urls)
    res = [content for content in contents if content["raw_content"] is not None]
    return res
data_from_links = run()
data_from_links


[{'url': 'https://www.foxweather.com/weather-news/burning-man-festival-nevada-death-flooding-monsoon',
  'raw_content': '\nDeath reported at Burning Man festival in Nevada as thousands of attendees get stranded by monsoon flooding | Fox Weather\n  Search Fox Weather  The FOX Weather App Learn More   \n              Watch Live\n            \n              Weather News\n            \n              How to Watch\n            \n              Extreme Weather\n            \n              Learn\n            \n              Local Weather\n            \n              Earth & Space\n            \n              Lifestyle\n            \n              Video\n            \n              America\'s Weather Team\n            \n              Podcast\n             Weather News  Published  \n        September 3, 2023 2:55pm EDT\n       \n            Death reported at Burning Man festival in Nevada as thousands of attendees get stranded by monsoon flooding\n           \n            Event organizers told th